In [13]:
import json 
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

## Loading openai key
with open('../environ.json') as f:
    config = json.load(f)
    api_keys = list(config.values())[0]

# assign openai key to an environment variable
os.environ["OPENAI_API_KEY"] = api_keys

# # check to confirm

In [14]:
# Define Neo4j connections
from neo4j import GraphDatabase
host = 'bolt://127.0.0.1:7687'
user = 'admin'
password = 'cran2graph'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query, params={}):
    with driver.session(database="cran") as session:
        result = session.run(query, params)
        return result.to_df()

In [15]:
# establish connection to neo4j graph database
graph = Neo4jGraph(
    url="bolt://127.0.0.1:7687", username="admin", password="cran2graph", database="cran"
)

In [16]:
# check the graph schema
print(graph.schema)


        Node properties are the following:
        [{'labels': 'Person', 'properties': [{'property': 'person', 'type': 'STRING'}]}, {'labels': 'Package', 'properties': [{'property': 'version', 'type': 'STRING'}, {'property': 'package', 'type': 'STRING'}, {'property': 'description', 'type': 'STRING'}, {'property': 'embedding', 'type': 'LIST'}, {'property': 'md5sum', 'type': 'STRING'}, {'property': 'license', 'type': 'STRING'}, {'property': 'published', 'type': 'STRING'}]}, {'labels': 'License', 'properties': [{'property': 'license', 'type': 'STRING'}]}, {'labels': 'Institution', 'properties': [{'property': 'institution', 'type': 'STRING'}]}, {'labels': 'Dependency', 'properties': [{'property': 'name', 'type': 'STRING'}]}]
        Relationship properties are the following:
        []
        The relationships are the following:
        ['(:Person)-[:CONTRIBUTED_TO]->(:Package)', '(:Person)-[:WORKS_IN]->(:Institution)', '(:Person)-[:MAINTAINS]->(:Package)', '(:Package)-[:DEPENDS_ON]->(:D

In [17]:
query = """

MATCH (p:Package)
MATCH (p)-[r:CONTRIBUTED_TO|MAINTAINS]-(n:Person)
WITH p, type(r) as type, collect(n.person) as names
WITH p, type+": "+reduce(s="", n IN names | s + n + ", ") as types
WITH p, collect(types) as contexts
WITH p, "Package Name: "+ p.package + " year: "+p.published +" Description: "+ p.description +"\n" +
        reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n") as context
RETURN context Limit 1

"""

In [15]:
run_query("""
CALL apoc.periodic.iterate(
    'MATCH (p:Package) RETURN id(p) as id',
    'MATCH (p:Package)
    WHERE id(p) = id
    MATCH (p)-[r:CONTRIBUTED_TO|MAINTAINS]-(n:Person)
    WITH p, type(r) as type, collect(n.person) as names
    WITH p, type+": "+reduce(s="", n IN names | s + n + ", ") as types
    WITH p, collect(types) as contexts
    WITH p, "Package Name: "+ p.package + " year: "+p.published +" Description: "+ p.description +"\n" +
            reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n") as context
    CALL apoc.ml.openai.embedding([context], $apiKey) YIELD embedding
    SET p.embedding = embedding',
    {batchSize:1, retries:3, params: {apiKey: $apiKey}})
""", {'apiKey': api_keys})['errorMessages'][0]
        
        

In [18]:
system_prompt = """
You are an assistant that helps to generate text to form nice and human understandable answers based.
The latest prompt contains the information, and you need to generate a human readable response based on the given information.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
Do not add any additional information that is not explicitly provided in the latest prompt.
I repeat, do not add any information that is not explicitly given.
"""

In [19]:
def generate_user_prompt(question, context):
    return f"""
    The question is {question}
    Answer the question by using the provided information:
    {context}
    """

In [20]:
def retrieve_context(question, k=3):
    data = run_query("""
        // retrieve the embedding of the question
        CALL apoc.ml.openai.embedding([$question], $apiKey) YIELD embedding 
        // match relevant packages
        MATCH (p:Package)
        WITH p, gds.similarity.cosine(embedding, p.embedding) AS score
        ORDER BY score DESC
        // LIMIT the number of relevant description
        LIMIT toInteger($k)
        // retrieve graph context 
        MATCH (p)--()--(p1:Package)
        WITH p, p1, count(*) AS count
        ORDER BY count DESC
        WITH p, apoc.text.join(collect(p1.package)[..3], ", ") as similarPackage
        MATCH (p)-[r:CONTRIBUTED_TO|MAINTAINS]-(n:Person)
        WITH p, similarPackage, type(r) as type, collect(n.person) as names
        WITH p, similarPackage, type+": "+reduce(s="", n IN names | s + n + ", ") as types
        WITH p, similarPackage, collect(types) as contexts
        WITH p, similarPackage, "Package Name: "+ p.package + " year: "+p.published +" Description: "+ p.description +"\n" +
                reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n")  + "similar packages:" + similarPackage + "\n" as context
        RETURN context  
    """, {'question': question, 'k': k, 'apiKey': api_keys})
    return data['context'].to_list()

In [21]:
def generate_answer(question):
    # Retrieve context
    context = retrieve_context(question)
    # Print context
    for c in context:
        print(c)
    # Generate answer
    response = run_query(
        """
  CALL apoc.ml.openai.chat([{role:'system', content: $system},
                      {role: 'user', content: $user}], $apiKey) YIELD value
  RETURN value.choices[0].message.content AS answer
  """,
        {
            "system": system_prompt,
            "user": generate_user_prompt(question, context),
            "apiKey": api_keys,
        },
    )
    return response["answer"][0]

In [22]:
generate_answer("Who maintains DBI package?")

Package Name: DatabaseConnector year: 2023-09-07 Description: An R DataBase Interface (DBI) compatible interface to various database platforms (PostgreSQL, Oracle, Microsoft SQL Server, 
    Amazon Redshift, Microsoft Parallel Database Warehouse, IBM Netezza, Apache Impala, Google BigQuery, Snowflake, Spark, and SQLite). Also includes support for
    fetching data as Andromeda objects. Uses either Java Database Connectivity (JDBC) or other DBI drivers to connect to databases.
CONTRIBUTED_TO: Amazon Inc., PostgreSQL Global Development Group, Oracle Inc., Microsoft Inc., Observational Health Data Science and Informatics, Marc Suchard, Martijn Schuemie
similar packages:DatabaseConnectorJars, CDMConnector, SqlRender

Package Name: RODBCDBI year: 2016-03-14 Description: An implementation of Rs DBI interface using ODBC package as a
    back-end. This allows R to connect to any DBMS that has a ODBC driver.
CONTRIBUTED_TO: Shinichi Takayanagi, Nagi Teramo
similar packages:rsyncrosim, odbc, dbg

'The DBI package is maintained by the contributors Andrew Kane, Marc Suchard, Martijn Schuemie, PostgreSQL Global Development Group, Oracle Inc., Microsoft Inc., and Observational Health Data Science and Informatics.'